# AHRI Health Informatics Internship – Data Analysis Assessment

**Candidate:** Simon Mufara  
**Date:** November 2025  
**Institution:** Africa Health Research Institute (AHRI)  
**Project:** Hypertension Clinic Attendance Analysis  
**Environment:** Python (pandas, seaborn, plotly, matplotlib)

---

### 🎯 Objective
This analysis explores:
1. Data quality and consistency across all provided datasets.  
2. Visualization of participant demographics and clinic attendance.  
3. Key insights into hypertension diagnosis and follow-up visits.

---

**Datasets Provided**
| File | Description |
|------|--------------|
| participants.csv | Participant demographics (ID, sex, DOB) |
| health_screen.csv | Health screening info (BP, BMI, smoking, hypertension diagnosis) |
| clinic_visits.csv | Clinic visit history and reasons |
| clinic_codes.csv | Clinic ID reference |


# Imports & Setup 

In [1]:
# 1. Imports & configuration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Visual config
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_context("talk")  # good sizes for presentations

# Paths
data_dir = Path(r"C:\Users\kayka\Downloads\healthinformaticsinternshipassessmentexercise")
output_dir = data_dir / "output_ahri_q1"
fig_dir = output_dir / "figures"
output_dir.mkdir(exist_ok=True)
fig_dir.mkdir(exist_ok=True)

# Confirm
print("Data dir:", data_dir)
print("Output dir:", output_dir)

# 2. Load raw data (do not modify original CSVs)
participants = pd.read_csv(data_dir / "participants.csv", dtype=str)
health_screen = pd.read_csv(data_dir / "health_screen.csv", dtype=str)
clinic_visits = pd.read_csv(data_dir / "clinic_visits.csv", dtype=str)
clinic_codes = pd.read_csv(data_dir / "clinic_codes.csv", dtype=str)

# Quick peek
print("Loaded shapes:")
print("participants:", participants.shape)
print("health_screen:", health_screen.shape)
print("clinic_visits:", clinic_visits.shape)
print("clinic_codes:", clinic_codes.shape)



Data dir: C:\Users\kayka\Downloads\healthinformaticsinternshipassessmentexercise
Output dir: C:\Users\kayka\Downloads\healthinformaticsinternshipassessmentexercise\output_ahri_q1
Loaded shapes:
participants: (844, 3)
health_screen: (844, 7)
clinic_visits: (3174, 3)
clinic_codes: (3164, 3)


# Data Quality Check 

In [2]:
# 3. DATA QUALITY CHECKS (automated) -> produce QC report file
qc_lines = []
def qcline(s): 
    qc_lines.append(s)
    print(s)

qcline("=== Basic column overview ===")
for name, df in [("participants", participants), ("health_screen", health_screen), ("clinic_visits", clinic_visits), ("clinic_codes", clinic_codes)]:
    qcline(f"{name}: {df.shape[0]} rows, {df.shape[1]} columns")
    qcline(f"  columns: {list(df.columns)}")

# Missing values per dataset
qcline("\n=== Missing values (per dataset) ===")
for name, df in [("participants", participants), ("health_screen", health_screen), ("clinic_visits", clinic_visits), ("clinic_codes", clinic_codes)]:
    miss = df.isna().sum()
    # keep only columns with missing
    miss = miss[miss>0]
    if miss.empty:
        qcline(f"{name}: No missing values detected.")
    else:
        qcline(f"{name}: {len(miss)} columns with missing values")
        for c, v in miss.items():
            qcline(f"  - {c}: {v} missing")

# Duplicate rows
qcline("\n=== Duplicate rows check ===")
for name, df in [("participants", participants), ("health_screen", health_screen), ("clinic_visits", clinic_visits)]:
    dup_count = df.duplicated().sum()
    qcline(f"{name}: {dup_count} duplicated full rows")

# ID integrity
qcline("\n=== ID integrity ===")
for name, df in [("participants", participants), ("health_screen", health_screen), ("clinic_visits", clinic_visits)]:
    if 'id_new' in df.columns:
        n_null = df['id_new'].isna().sum()
        n_dup = df['id_new'].duplicated().sum()
        qcline(f"{name}: id_new missing={n_null}, id_new duplicates={n_dup}")
    else:
        qcline(f"{name}: id_new column MISSING")

# Save QC report
qc_path = output_dir / "QC_issues.txt"
with open(qc_path, "w") as fh:
    fh.write("\n".join(qc_lines))
qcline(f"\nQC summary written to: {qc_path}")


=== Basic column overview ===
participants: 844 rows, 3 columns
  columns: ['id_new', 'sex', 'dateofbirth']
health_screen: 844 rows, 7 columns
  columns: ['id_new', 'date_screen', 'systolicBP', 'diastolicBP', 'bmi', 'smokecat', 'BPdiag']
clinic_visits: 3174 rows, 3 columns
  columns: ['visitreason', 'id_new', 'visitdate']
clinic_codes: 3164 rows, 3 columns
  columns: ['id_new', 'visitdate', 'clinic_code']

=== Missing values (per dataset) ===
participants: No missing values detected.
health_screen: No missing values detected.
clinic_visits: No missing values detected.
clinic_codes: No missing values detected.

=== Duplicate rows check ===
participants: 0 duplicated full rows
health_screen: 0 duplicated full rows
clinic_visits: 8 duplicated full rows

=== ID integrity ===
participants: id_new missing=0, id_new duplicates=0
health_screen: id_new missing=0, id_new duplicates=0
clinic_visits: id_new missing=0, id_new duplicates=2666

QC summary written to: C:\Users\kayka\Downloads\healthin

# Data Cleaning

In [3]:
# 4. CLEANING & STANDARDIZATION
# - Normalize column names to lowercase & strip spaces
def normalize_cols(df):
    df = df.copy()
    df.columns = df.columns.str.lower().str.strip()
    return df

participants = normalize_cols(participants)
health_screen = normalize_cols(health_screen)
clinic_visits = normalize_cols(clinic_visits)
clinic_codes = normalize_cols(clinic_codes)

# Ensure id_new exists and is string normalized
for df_name, df in [("participants", participants), ("health_screen", health_screen), ("clinic_visits", clinic_visits)]:
    if 'id_new' not in df.columns:
        raise ValueError(f"id_new missing in {df_name}")
    df['id_new'] = df['id_new'].astype(str).str.strip()

# Flexible date parsing (try common formats)
def parse_dates(df, candidates):
    for col in candidates:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], dayfirst=True, errors='coerce')
    return df

participants = parse_dates(participants, ['date of birth','dateofbirth','dob'])
health_screen = parse_dates(health_screen, ['date_screen','date screen','date'])
clinic_visits = parse_dates(clinic_visits, ['visitdate','visit date','date'])

# Standardize text columns
for col in ['bpdiag','smokecat','sex','visitreason']:
    for df in [participants, health_screen, clinic_visits]:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().replace({'nan': np.nan})
            df[col] = df[col].str.lower()

# Clean visitreason small fixes (remove excessive whitespace)
if 'visitreason' in clinic_visits.columns:
    clinic_visits['visitreason'] = clinic_visits['visitreason'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip().replace({'nan': np.nan})

print("Cleaning complete. Sample columns now:")
print("participants columns:", participants.columns.tolist())
print("health_screen columns:", health_screen.columns.tolist())
print("clinic_visits columns:", clinic_visits.columns.tolist())


Cleaning complete. Sample columns now:
participants columns: ['id_new', 'sex', 'dateofbirth']
health_screen columns: ['id_new', 'date_screen', 'systolicbp', 'diastolicbp', 'bmi', 'smokecat', 'bpdiag']
clinic_visits columns: ['visitreason', 'id_new', 'visitdate']


# Data Integration 

In [4]:
# 5. MERGE DATASETS for analysis convenience
merged = health_screen.merge(participants, on="id_new", how="left", suffixes=('_screen','_part'))
merged_visits = clinic_visits.merge(merged, on="id_new", how="left")

print("Merged shapes:", merged.shape, merged_visits.shape)


Merged shapes: (844, 9) (3174, 11)


# Analysis 

In [5]:
# 6. ANALYSIS QUESTIONS (a-d)
# a) How many participants diagnosed with hypertension attended a clinic?

# find bpdiag column robustly
bp_cols = [c for c in health_screen.columns if 'bpdiag' in c]
if not bp_cols:
    raise ValueError("No BPdiag column found in health_screen")
bp_col = bp_cols[0]

hypertensive_ids = health_screen[health_screen[bp_col].str.lower() == 'yes']['id_new'].unique()
total_hypertensive = len(hypertensive_ids)

# clinic visits by hypertensives
hypertensive_visits = clinic_visits[clinic_visits['id_new'].isin(hypertensive_ids)]
hypertensive_attended_count = hypertensive_visits['id_new'].nunique()

# b) How many had hypertension as the reason for visit?
hyp_keywords = ["hypertension","hyperten","high blood","blood pressure","bp"]
# build pattern
pattern = "|".join(hyp_keywords)
clinic_visits['reason_hyp_flag'] = clinic_visits['visitreason'].fillna("").str.contains(pattern, case=False, regex=True)
hyp_reason_count = clinic_visits[clinic_visits['reason_hyp_flag'] == True]['id_new'].nunique()

# c) How many other (non-hypertensive) participants attended a clinic?
non_hyp_ids = health_screen[~health_screen['id_new'].isin(hypertensive_ids)]['id_new'].unique()
non_hyp_attended_count = clinic_visits[clinic_visits['id_new'].isin(non_hyp_ids)]['id_new'].nunique()

# d) Most common reasons (top 10)
top_reasons = clinic_visits['visitreason'].value_counts().head(10).rename_axis('reason').reset_index(name='count')

# Print results
print("===== ANALYSIS SUMMARY =====")
print(f"Total hypertensive participants (diagnosed at screen): {total_hypertensive}")
print(f"(a) Hypertensive participants who attended clinic: {hypertensive_attended_count}")
print(f"(b) Participants with hypertension-related visit reason: {hyp_reason_count}")
print(f"(c) Non-hypertensive participants who attended clinic: {non_hyp_attended_count}")
print("\n(d) Top 10 visit reasons:\n", top_reasons.to_string(index=False))

# 7. Save summary table and QC details
summary_df = pd.DataFrame({
    'metric': [
        'total_hypertensive',
        'hypertensive_attended',
        'hypertension_reason_count',
        'non_hypertensive_attended'
    ],
    'value': [
        total_hypertensive,
        hypertensive_attended_count,
        hyp_reason_count,
        non_hyp_attended_count
    ]
})
summary_df.to_csv(output_dir / "analysis_summary.csv", index=False)

# Append a brief QC summary to the QC file
with open(qc_path, 'a') as fh:
    fh.write("\n\n=== Analysis summary ===\n")
    fh.write(summary_df.to_string(index=False))
print("Saved analysis_summary.csv and updated QC_issues.txt")



===== ANALYSIS SUMMARY =====
Total hypertensive participants (diagnosed at screen): 128
(a) Hypertensive participants who attended clinic: 85
(b) Participants with hypertension-related visit reason: 101
(c) Non-hypertensive participants who attended clinic: 423

(d) Top 10 visit reasons:
                       reason  count
chronic care - art follow-up   1512
 chronic care - hypertension    502
 minor ailment - first visit    483
       maternity - antenatal    161
   minor ailment - follow-up    117
        chronic care - other     88
 family planning - follow-up     84
        chronic care - ccmdd     62
     chronic care - diabetes     59
chronic care - mental health     32
Saved analysis_summary.csv and updated QC_issues.txt


# Data Visualisation 

In [6]:
# 8. VISUALISATIONS - Static (Seaborn/Matplotlib) and interactive (Plotly)
# 8.1 Age distribution (if date of birth present)
if any('birth' in c for c in merged_visits.columns):
    dob_col = [c for c in merged_visits.columns if 'birth' in c][0]
    merged_visits['age'] = (pd.Timestamp("2019-12-31") - merged_visits[dob_col]).dt.days / 365.25
    age_series = merged_visits['age'].dropna()
    if not age_series.empty:
        plt.figure(figsize=(10,5))
        sns.histplot(age_series, bins=20, kde=True, color="#2b8cbe")
        plt.title("Age distribution of participants", fontsize=16, fontweight='bold')
        plt.xlabel("Age (years)")
        plt.ylabel("Count")
        plt.tight_layout()
        plt.savefig(fig_dir / "age_distribution.png", dpi=300)
        plt.savefig(fig_dir / "age_distribution.svg")
        plt.close()

        # Interactive
        fig = px.histogram(merged_visits, x='age', nbins=25, title="Age distribution (interactive)")
        fig.update_layout(template="plotly_white")
        fig.write_html(str(fig_dir / "age_distribution_interactive.html"))


In [7]:
# 8.2 Top visit reasons - horizontal bar (static + interactive)
if not top_reasons.empty:
    plt.figure(figsize=(10,6))
    sns.barplot(x='count', y='reason', data=top_reasons, palette='viridis')
    plt.title("Top 10 reasons for clinic visits", fontsize=16, fontweight='bold')
    plt.xlabel("Number of visits")
    plt.ylabel("Visit reason")
    plt.tight_layout()
    plt.savefig(fig_dir / "top_10_visit_reasons.png", dpi=300)
    plt.savefig(fig_dir / "top_10_visit_reasons.svg")
    plt.close()

    # interactive
    fig = px.bar(top_reasons.sort_values('count'), x='count', y='reason', orientation='h',
                 title="Top 10 reasons for clinic visits (interactive)", labels={'count':'Number of visits','reason':'Reason'})
    fig.update_layout(template="plotly_white", height=500)
    fig.write_html(str(fig_dir / "top_10_visit_reasons_interactive.html"))


In [8]:
# 8.3 Hypertension attendance - pie + bar (static + interactive)
attendance_df = pd.DataFrame({
    'status': ['attended','did_not_attend'],
    'count': [hypertensive_attended_count, max(0, total_hypertensive - hypertensive_attended_count)]
})
# static pie via matplotlib
plt.figure(figsize=(6,6))
plt.pie(attendance_df['count'], labels=attendance_df['status'], autopct='%1.1f%%', colors=['#2ecc71', '#e74c3c'], startangle=140)
plt.title("Clinic attendance among hypertensive participants", fontsize=14, fontweight='bold')
plt.savefig(fig_dir / "hypertension_attendance_pie.png", dpi=300)
plt.savefig(fig_dir / "hypertension_attendance_pie.svg")
plt.close()

# interactive pie
fig = px.pie(attendance_df, names='status', values='count', title='Clinic attendance among hypertensive participants', hole=0.4)
fig.update_traces(textinfo='percent+label')
fig.write_html(str(fig_dir / "hypertension_attendance_interactive.html"))


In [9]:
# 8.4 Sunburst: bp status -> sex -> top reason (interactive)
# prepare small table with categories (limit reason cardinality)
merged_visits['bp_status'] = np.where(merged_visits[bp_col] == 'yes', 'Hypertensive', 'Non-hypertensive')
merged_visits['visitreason_cat'] = merged_visits['visitreason'].fillna("unknown")
# collapse low frequency reasons into 'other' to keep sunburst readable
reason_counts_all = merged_visits['visitreason_cat'].value_counts()
top_reasons_list = reason_counts_all.head(20).index.tolist()
merged_visits['visitreason_cat2'] = merged_visits['visitreason_cat'].where(merged_visits['visitreason_cat'].isin(top_reasons_list), 'other')

fig = px.sunburst(
    merged_visits,
    path=['bp_status', 'sex', 'visitreason_cat2'],
    title='Clinic visits: hypertension status → sex → reason (top reasons & other)',
)
fig.update_layout(margin=dict(t=40, l=0, r=0, b=0))
fig.write_html(str(fig_dir / "sunburst_visits_bp_sex_reason.html"))


In [10]:
merged_visits["bp_status"] = np.where(merged_visits["bpdiag"] == "yes", "Hypertensive", "Non-Hypertensive")

fig = px.sunburst(
    merged_visits,
    path=["bp_status", "sex", "visitreason"],
    title="Visit Breakdown by Hypertension Status, Sex, and Reason",
    color_discrete_sequence=px.colors.qualitative.Set3
)
fig.show()


#  Summary of Findings

| Question | Answer |
|-----------|---------|
| a. Hypertensive participants who attended clinic | **85** |
| b. Hypertension as reason for visit | **101** |
| c. Other participants who attended | **423** |
| d. Most common reasons for visits | *Follow up, Hypertension, & minor alinment Checkups*|

---

###  Key Insights
- Data quality was generally good, with minimal missing values.
- Majority of clinic visits were for routine checkups or chronic care.
- A significant proportion of hypertensive individuals followed up at clinics — indicating good referral adherence.
- Interactive plots highlight demographic and clinical patterns suitable for reporting or presentation.

---

**Next Steps:**
- Automate this workflow using a Jupyter/Streamlit dashboard.
- Integrate geographic or temporal visualizations for deeper insights.
